# An Earley parser
All code is by Wikler Aziz unless noted explicitly. New are: earley_axioms, predict and earley.

This notebook essentially builts on the notebook lab-cky.ipynb, and copied code from lab-cfg.ipynb.

In [2]:
from cfg import read_grammar_rules, WCFG
from rule import Rule
from symbol import is_terminal, is_nonterminal, make_symbol
from collections import defaultdict
from item import Item
from agenda import Agenda

In [53]:
def cky_axioms(cfg, sentence):
    """
    :params cfg: a context-free grammar (an instance of WCFG)
    :params sentence: the input sentence (as a list or tuple)
    :returns: a list of items
    """
    items = []
    for rule in cfg:
        for i in range(len(sentence)):
            items.append(Item(rule, [i]))
    return items

def earley_axioms(cfg, sentence, start):
    """
    Author: Daan
    Axioms for Earley.

    Inference rule:
        -------------------- (S -> alpha) in cfgs
        [S -> * alpha, [0]] 

    :param cfg: a context-free grammar (an instance of WCFG)
    :param sentence: the input sentence (as a list or tuple)
    :returns: a list of items that are Earley axioms  
    """
    items = []
    for rule in cfg.get(start):
        items.append(Item(rule, [0]))
    return items

def predict(cfg, item):
    """
    Author: Daan
    Prediction for Earley.

    Inference rule:
        -------------------- (SS -> alpha) in cfgs
        [S -> * alpha, [0]] 

    :param cfg: a context-free grammar (an instance of WCFG)
    :param item: an active Item
    :returns: a list of predicted Items or None  
    """
    items = []
    rules = cfg.get(item.next)
    for rule in rules:
            items.append(Item(rule, [item.dot]))
    return items

def scan(item, sentence):
    """
    Scan a terminal (compatible with CKY and Earley).
    
    Inference rule:
    
        [X -> alpha * x beta, [i ... j]]
        ------------------------------------    sentence[j] == x
        [X -> alpha x * beta, [i ... j + 1]]
    
    :param item: an active Item
    :param sentence: a list/tuple of terminals
    :returns: an Item or None
    """
    assert is_terminal(item.next), 'Only terminal symbols can be scanned, got %s' % item.next
    if item.dot < len(sentence) and sentence[item.dot] == item.next:
        return item.advance(item.dot + 1)
    else:
        return None
    
def complete(item, agenda):
    """
    Move dot over nonterminals (compatible with CKY and Earley).
    
    Inference rule:
    
        [X -> alpha * Y beta, [i ... k]] [Y -> gamma *, [k ... j]]
        ----------------------------------------------------------
                 [X -> alpha Y * beta, [i ... j]]
                 
    :param item: an active Item.
        if `item` is complete, we advance the dot of incomplete passive items to `item.dot`
        otherwise, we check whether we know a set of positions J = {j1, j2, ..., jN} such that we can
        advance this item's dot to.
    :param agenda: an instance of Agenda
    :returns: a list of items
    """
    items = []
    if item.is_complete():
        # advance the dot for incomplete items waiting for item.lhs spanning from item.start
        for incomplete in agenda.waiting(item.lhs, item.start):
            items.append(incomplete.advance(item.dot))
    else:
        # look for completions of item.next spanning from item.dot
        ends = set()
        for complete in agenda.complete(item.next, item.dot):
            ends.add(complete.dot)
        # advance the dot of the input item for each position that complete a span
        for end in ends:
            items.append(item.advance(end))
    return items


def make_forest(complete_items):
    """
    Turn complete items into a WCFG.
    
    :param complete_items: complete items (iterable)
    :returns: a WCFG
    """
    forest = WCFG()
    for item in complete_items:
        lhs = make_symbol(item.lhs, item.start, item.dot)
        rhs = []
        for i, sym in enumerate(item.rule.rhs):
            rhs.append(make_symbol(sym, item.state(i), item.state(i + 1)))
        forest.add(Rule(lhs, rhs, item.rule.prob))
    return forest

def make_chart(complete_items, n):
    chart = [[defaultdict(list) for j in range(n)] for i in range(n)] # n by n matrix with edges
    for item in complete_items:
        chart[item.start][item.dot][item.lhs].append((item.rule, item.dots_))
    return chart

## CKY parser

In [15]:
def cky(cfg, sentence):
    A = Agenda()
    for item in cky_axioms(cfg, sentence):
        A.push(item)
    while A:
        item = A.pop()
        if item.is_complete() or is_nonterminal(item.next):
            for new in complete(item, A):
                A.push(new)
        else:
            new = scan(item, sentence)
            if new is not None:
                A.push(new)
        A.make_passive(item)
    return make_forest(A.itercomplete())

## Earley parser

In [52]:
def earley(cfg, sentence, start):
    """
    Author: Daan
    """
    A = Agenda()
    for item in earley_axioms(cfg, sentence, start):
        A.push(item)
    while len(A) > 0:
        item = A.pop()
        if item.is_complete() or is_nonterminal(item.next):
            for new in predict(cfg, item):
                A.push(new)
            for new in complete(item, A):
                A.push(new)
        else:
            new = scan(item, sentence)
            if new is not None:
                A.push(new)
        A.make_passive(item)
    return make_forest(A.itercomplete())

## Comparing parsers

We can compare the Earley and CKY parsers on a number of different grammars.

In [17]:
# with this we can plot trees
from util import make_nltk_tree

Our first grammar is the grammar of arithmetic expressions involving sum an multiplication. This grammar is taken from Earley's original 1970 paper.

In [26]:
G1 = WCFG(read_grammar_rules(open('examples/arithmetic', 'r')))
print G1

[T] -> [P] (0.5)
[T] -> [T] * [P] (0.5)
[E] -> [T] (0.5)
[E] -> [E] + [T] (0.5)
[P] -> a (1.0)


Then we analyze an ambiguous version of that grammar:

In [49]:
G2 = WCFG(read_grammar_rules(open('examples/ambiguous', 'r')))
print G2

[T] -> [P] (0.5)
[T] -> [T] * [P] (0.4)
[T] -> [T] + [P] (0.1)
[E] -> [T] (0.5)
[E] -> [E] + [T] (0.45)
[E] -> [E] * [T] (0.05)
[P] -> a (1.0)


Lastly we consider a grammar that produces natural language sentences:

In [58]:
G3 = WCFG(read_grammar_rules(open('examples/hdp-pcfg-grammar', 'r')))
print G3

[NN] -> mouse (0.33)
[NN] -> cat (0.33)
[NN] -> dog (0.34)
[JJ] -> big (0.5)
[JJ] -> black (0.5)
[DT] -> the (0.5)
[DT] -> a (0.5)
[NPBAR] -> [JJ] [NN] (0.5)
[NPBAR] -> [JJ] [NPBAR] (0.5)
[VP] -> [VB] [NP] (1.0)
[S] -> [NP] [VP] (1.0)
[VB] -> chased (0.5)
[VB] -> ate (0.5)
[NP] -> [DT] [NN] (0.5)
[NP] -> [DT] [NPBAR] (0.5)


In [62]:
import numpy as np

def generate_sample(grammar, items=('[E]',)):
    """
    Given a grammar returns a sentence from it using
    the probabilities specfied in the grammar.
    :param items: call the function with (start,) where 
                  start is the start symbol of the grammar
    :returns: a sentence from the language as a list
    """
    frags = []
    for item in items:
        if is_nonterminal(item):
            productions = grammar.get(item)
            ps = [production.prob for production in productions]
            random_index = np.argmax(np.random.multinomial(1, ps, size=1))
            prod = productions[random_index]
            frags.extend(generate_sample(grammar, items=prod.rhs))
        else:
            frags.append(item)
    return frags

In [79]:
sentence1 = generate_sample(G1)
print sentence1

['a', '*', 'a', '+', 'a']


In [94]:
goal = make_symbol('[E]', 0, len(sentence1))

forest = cky(G1, sentence1)
print forest
print len(forest)

forest = earley(G1, sentence1, goal)
print forest
print len(forest)

[E:2-3] -> [T:2-3] (0.5)
[E:4-5] -> [T:4-5] (0.5)
[T:0-3] -> [T:0-1] * [P:2-3] (0.4)
[T:0-5] -> [T:0-3] + [P:4-5] (0.1)
[E:0-5] -> [E:0-3] + [T:4-5] (0.45)
[E:0-5] -> [T:0-5] (0.5)
[E:0-5] -> [E:0-1] * [T:2-5] (0.05)
[T:0-1] -> [P:0-1] (0.5)
[E:0-1] -> [T:0-1] (0.5)
[E:0-3] -> [E:0-1] * [T:2-3] (0.05)
[E:0-3] -> [T:0-3] (0.5)
[E:2-5] -> [T:2-5] (0.5)
[E:2-5] -> [E:2-3] + [T:4-5] (0.45)
[P:0-1] -> a (1.0)
[T:4-5] -> [P:4-5] (0.5)
[P:2-3] -> a (1.0)
[T:2-5] -> [T:2-3] + [P:4-5] (0.1)
[T:2-3] -> [P:2-3] (0.5)
[P:4-5] -> a (1.0)
19

0


In [96]:
goal = make_symbol('[E]', 0, len(sentence1))

forest = cky(G2, sentence1)
print forest
print len(forest)

forest = earley(G2, sentence1, goal)
print forest
print len(forest)

[E:2-3] -> [T:2-3] (0.5)
[E:4-5] -> [T:4-5] (0.5)
[T:0-3] -> [T:0-1] * [P:2-3] (0.4)
[T:0-5] -> [T:0-3] + [P:4-5] (0.1)
[E:0-5] -> [E:0-3] + [T:4-5] (0.45)
[E:0-5] -> [T:0-5] (0.5)
[E:0-5] -> [E:0-1] * [T:2-5] (0.05)
[T:0-1] -> [P:0-1] (0.5)
[E:0-1] -> [T:0-1] (0.5)
[E:0-3] -> [E:0-1] * [T:2-3] (0.05)
[E:0-3] -> [T:0-3] (0.5)
[E:2-5] -> [T:2-5] (0.5)
[E:2-5] -> [E:2-3] + [T:4-5] (0.45)
[P:0-1] -> a (1.0)
[T:4-5] -> [P:4-5] (0.5)
[P:2-3] -> a (1.0)
[T:2-5] -> [T:2-3] + [P:4-5] (0.1)
[T:2-3] -> [P:2-3] (0.5)
[P:4-5] -> a (1.0)
19

0


In [84]:
sentence3 = generate_sample(G3, items=('[S]',))
print sentence3

['the', 'black', 'dog', 'chased', 'the', 'big', 'mouse']


In [99]:
goal = make_symbol('[S]', 0, len(sentence3))

forest = cky(G3, sentence3)
print forest
print len(forest)

forest = earley(G3, sentence3, goal)
print forest
print len(forest)

[NN:6-7] -> mouse (0.33)
[NN:2-3] -> dog (0.34)
[NP:0-3] -> [DT:0-1] [NPBAR:1-3] (0.5)
[DT:0-1] -> the (0.5)
[S:0-7] -> [NP:0-3] [VP:3-7] (1.0)
[NPBAR:5-7] -> [JJ:5-6] [NN:6-7] (0.5)
[NPBAR:1-3] -> [JJ:1-2] [NN:2-3] (0.5)
[DT:4-5] -> the (0.5)
[VP:3-7] -> [VB:3-4] [NP:4-7] (1.0)
[VB:3-4] -> chased (0.5)
[NP:4-7] -> [DT:4-5] [NPBAR:5-7] (0.5)
[JJ:1-2] -> black (0.5)
[JJ:5-6] -> big (0.5)
13

0
